# ⭐**Designing Agentic Systems with LangChain**

This notebook demonstrates how to integrate and dynamically route multiple tools in LangGraph, enabling flexible and memory-aware educational chatbot workflows.


## Table of Contents

- [***1. Introduction to Multiple Tools***](#1-introduction-to-multiple-tools)
- [***2. Custom Tools: Palindrome and Historical Dates***](#2-custom-tools-palindrome-and-historical-dates)
- [***3. Binding Multiple Tools***](#3-binding-multiple-tools)
- [***4. Building a Flexible Workflow***](#4-building-a-flexible-workflow)
- [***5. Streaming and Memory Integration***](#5-streaming-and-memory-integration)


In [ ]:
# Setup: Required modules
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_community.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage
from IPython.display import Image, display



## 1. Introduction to Multiple Tools

LangGraph allows agents to use multiple tools and determine the correct one based on user input.

- Automate tool selection per query
- Support multi-functional bots (e.g., palindrome checkers and historical lookup bots)
- Enable natural language control over which function is executed



## 2. Custom Tools: Palindrome and Historical Dates

Define custom tools using the `@tool` decorator for clarity and control.


In [ ]:
# Tool: Historical date lookup using LLM
@tool
def date_checker(date: str) -> str:
    """Provide a list of important historical events for a given date."""
    try:
        answer = llm.invoke(f"List important historical events that occurred on {date}.")
        return answer.content
    except Exception as e:
        return f"Error retrieving events: {str(e)}"

# Tool: Palindrome check using Python logic
@tool
def check_palindrome(text: str):
    """Check if a word or phrase is a palindrome."""
    cleaned = ''.join(char.lower() for char in text if char.isalnum())
    if cleaned == cleaned[::-1]:
        return f"The phrase or word '{text}' is a palindrome."
    else:
        return f"The phrase or word '{text}' is not a palindrome."



## 3. Binding Multiple Tools

Bind tools to a ToolNode and connect to the language model.


In [ ]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini", api_key="OPENAI_API_KEY")

# Bundle all tools
tools = [date_checker, check_palindrome]
tool_node = ToolNode(tools)
model_with_tools = llm.bind_tools(tools)



## 4. Building a Flexible Workflow

Define graph logic to handle both LLM-only responses and tool-based outputs.


In [ ]:
# Stopping condition: route based on tool call presence
def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    return "tools" if last_message.tool_calls else END

# Main model function: decides between LLM response or tool return
def call_model(state: MessagesState):
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return {"messages": [AIMessage(content=last_message.tool_calls[0]["response"])]}
    return {"messages": [model_with_tools.invoke(state["messages"])]}

# Build the full graph
workflow = StateGraph(MessagesState)
workflow.add_node("chatbot", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges("chatbot", should_continue, ["tools", END])
workflow.add_edge("tools", "chatbot")



## 5. Streaming and Memory Integration

Enable memory for multi-turn conversations and test with dynamic tool usage.


In [ ]:
# Add memory and compile the app
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
display(Image(app.get_graph().draw_mermaid_png()))


In [ ]:
# Function to stream responses
config = {"configurable": {"thread_id": "1"}}

def multi_tool_output(query):
    inputs = {"messages": [HumanMessage(content=query)]}
    for msg, metadata in app.stream(inputs, config, stream_mode="messages"):
        if msg.content and not isinstance(msg, HumanMessage):
            print(msg.content, end="", flush=True)
    print("\n")

# Example tool usage
multi_tool_output("Is `Stella won no wallets` a palindrome?")
multi_tool_output("What happened on April 12th, 1955?")


In [ ]:
# Follow-up multi-turn conversation
def user_agent_multiturn(queries):
    for query in queries:
        print(f"User: {query}")
        print("Agent: " + "".join(
            msg.content for msg, metadata in app.stream(
                {"messages": [HumanMessage(content=query)]},
                config,
                stream_mode="messages"
            ) if msg.content and not isinstance(msg, HumanMessage)
        ) + "\n")

queries = [
    "What happened on the 12 April 1961?",
    "What about 10 December 1948?",
    "Is `Mr. Owl ate my metal worm?` a palindrome?",
    "What about 'palladium stadium?'"
]
user_agent_multiturn(queries)
